In [1]:
library(data.table)
library(qvalue)

In [2]:
## RA (rheumathoid arthritis) SMR (summarised Mendelian Randomisation) results

In [3]:
# Matrix eQTL results
matrix_eqtl_smr_results_dir = "/directflow/SCCGGroupShare/projects/angxue/proj/SAIGE-eQTL/SMR/"
matrix_ibd_file = paste0(matrix_eqtl_smr_results_dir, "ra_all_14_celltypes_all_smr.txt")
matrix_df = fread(matrix_ibd_file)
head(matrix_df,2)

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,p_GWAS,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
FCRL5,1,FCRL5,157483167,rs12741984,1,157534818,G,A,0.787234,⋯,0.1312,0.113780,0.00486738,7.500497e-121,-0.236422,0.156769,0.1315321,0.0344273,20,BimmNaive
RPS8,1,RPS8,45240923,rs12120833,1,45241285,C,G,0.850097,⋯,0.1065,-0.105432,0.00500701,1.980368e-98,-0.385083,0.238770,0.1067927,0.6779076,20,BimmNaive


In [4]:
# Calculate number of unique genes, and combination of genes + celltypes
length(unique(matrix_df$Gene))
matrix_df$comb = paste0(matrix_df$Gene,"_",matrix_df$Cell_type)
length(unique(matrix_df$comb))
head(matrix_df,2)

[1] 6094

[1] 14526

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type,comb
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
FCRL5,1,FCRL5,157483167,rs12741984,1,157534818,G,A,0.787234,⋯,0.113780,0.00486738,7.500497e-121,-0.236422,0.156769,0.1315321,0.0344273,20,BimmNaive,FCRL5_BimmNaive
RPS8,1,RPS8,45240923,rs12120833,1,45241285,C,G,0.850097,⋯,-0.105432,0.00500701,1.980368e-98,-0.385083,0.238770,0.1067927,0.6779076,20,BimmNaive,RPS8_BimmNaive


In [5]:
nrow(matrix_df[matrix_df$p_SMR < 0.05/length(unique(matrix_df$Gene)),])
nrow(matrix_df[matrix_df$p_SMR < 0.05/length(unique(matrix_df$comb)),])

[1] 209

[1] 189

In [6]:
# SAIGE-QTL results
saige_eqtl_smr_results_dir = "/directflow/SCCGGroupShare/projects/angxue/proj/SAIGE-eQTL/SMR/saige_eQTL/"
saige_ibd_file = paste0(saige_eqtl_smr_results_dir, "ra_all_14_celltypes_all_smr.txt")
saige_df = fread(saige_ibd_file)
head(saige_df,2)

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,p_GWAS,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
NOC2L,1,NOC2L,879584,rs3748595,1,887560,A,C,0.0672147,⋯,0.4313,-0.153020,0.0352961,1.455500e-05,0.276434,0.35668,0.4383276,0.3042242,3,BimmNaive
C1orf86,1,C1orf86,2115903,rs11587831,1,2110848,T,G,0.7853000,⋯,0.6300,-0.115809,0.0148762,6.980209e-15,-0.115708,0.24051,0.6304503,0.2562137,8,BimmNaive


In [7]:
length(unique(saige_df$Gene))
saige_df$comb = paste0(saige_df$Gene,"_",saige_df$Cell_type)
length(unique(saige_df$comb))
head(saige_df,2)

[1] 5672

[1] 16417

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type,comb
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
NOC2L,1,NOC2L,879584,rs3748595,1,887560,A,C,0.0672147,⋯,-0.153020,0.0352961,1.455500e-05,0.276434,0.35668,0.4383276,0.3042242,3,BimmNaive,NOC2L_BimmNaive
C1orf86,1,C1orf86,2115903,rs11587831,1,2110848,T,G,0.7853000,⋯,-0.115809,0.0148762,6.980209e-15,-0.115708,0.24051,0.6304503,0.2562137,8,BimmNaive,C1orf86_BimmNaive


In [8]:
# significant results at p<0.05/M with M either # genes or # gene-celltype combinations
nrow(saige_df[saige_df$p_SMR < 0.05/length(unique(saige_df$Gene)),])
nrow(saige_df[saige_df$p_SMR < 0.05/length(unique(saige_df$comb)),])

[1] 249

[1] 216

In [9]:
### p < 0.05/M with M-number of unique gene-celltype combinations tested by both methods

In [10]:
matrix_combs = unique(matrix_df$comb)
saige_combs = unique(saige_df$comb)
common_combs = saige_combs[saige_combs %in% matrix_combs]
length(common_combs)

[1] 11988

In [11]:
matrix_df_common = matrix_df[matrix_df$comb %in% common_combs,]
nrow(matrix_df_common)
nrow(matrix_df_common[matrix_df_common$p_SMR < (0.05/length(common_combs)),])

[1] 11988

[1] 188

In [12]:
saige_df_common = saige_df[saige_df$comb %in% common_combs,]
nrow(saige_df_common)
nrow(saige_df_common[saige_df_common$p_SMR < (0.05/length(common_combs)),])

[1] 11988

[1] 207

In [13]:
# significant results at FDR<5% instead (using qvalue)
matrix_df_common$qv = qvalue(matrix_df_common$p_SMR)$qvalues
nrow(matrix_df_common[matrix_df_common$qv<0.05,])

[1] 487

In [14]:
saige_df_common$qv = qvalue(saige_df_common$p_SMR)$qvalues
nrow(saige_df_common[saige_df_common$qv<0.05,])

[1] 529

In [15]:
# Using p < (0.05 / n_tests, gene-celltype comb)

matrix_sign_combs = matrix_df_common[matrix_df_common$p_SMR < (0.05/length(common_combs)),"comb"]$comb
length(matrix_sign_combs)
head(matrix_sign_combs)

saige_sign_combs = saige_df_common[saige_df_common$p_SMR < (0.05/length(common_combs)),"comb"]$comb
length(saige_sign_combs)
head(saige_sign_combs)

length(saige_sign_combs[saige_sign_combs %in% matrix_sign_combs])

[1] 188

[1] "REL_BimmNaive"                "HLA-DQA2_BimmNaive"          
[3] "HLA-DRB5_BimmNaive"           "XXbac-BPG299F13.17_BimmNaive"
[5] "HLA-DRB1_BimmNaive"           "HLA-DQB1_BimmNaive"

[1] 207

[1] "REL_BimmNaive"                "HLA-A_BimmNaive"             
[3] "XXbac-BPG299F13.17_BimmNaive" "HLA-C_BimmNaive"             
[5] "LST1_BimmNaive"               "C6orf48_BimmNaive"

[1] 179

In [16]:
# Using FDR < 0.05

matrix_sign_combs = matrix_df_common[matrix_df_common$qv < 0.05,"comb"]$comb
length(matrix_sign_combs)

saige_sign_combs = saige_df_common[saige_df_common$qv < 0.05,"comb"]$comb
length(saige_sign_combs)

length(saige_sign_combs[saige_sign_combs %in% matrix_sign_combs])

[1] 487

[1] 529

[1] 466

In [35]:
## SAIGE-QTL SMR gene list
saige_df_sign = saige_df_common[saige_df_common$p_SMR < (0.05/length(common_combs)),]
saige_df_sign

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type,comb,qv
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>
ERAP2,5,ERAP2,96211643,rs27295,5,96358687,C,T,0.431335,⋯,0.02186180,9.580355e-47,-0.1840280,0.03696720,6.419558e-07,6.248309e-02,20,BimmNaive,ERAP2_BimmNaive,7.135444e-05
LST1,6,LST1,31553901,rs2256965,6,31555130,A,G,0.405706,⋯,0.02502590,4.159973e-16,-0.3190970,0.06625470,1.463083e-06,9.578130e-02,14,BimmNaive,LST1_BimmNaive,1.472176e-04
HLA-DQA1,6,HLA-DQA1,32595956,rs9272546,6,32606941,T,C,0.336074,⋯,0.01390830,2.408243e-45,0.4690740,0.06514700,6.010586e-13,3.898305e-10,20,BimmNaive,HLA-DQA1_BimmNaive,2.498063e-10
HLA-DQB1,6,HLA-DQB1,32627244,rs9274660,6,32636434,A,G,0.394584,⋯,0.01463280,3.180543e-58,-0.3397790,0.05138470,3.780030e-11,3.378640e-09,20,BimmNaive,HLA-DQB1_BimmNaive,1.129171e-08
HLA-DQA2,6,HLA-DQA2,32709119,rs9272319,6,32604108,T,C,0.653288,⋯,0.07241990,2.105526e-94,0.0763340,0.00850834,2.920490e-19,1.322820e-06,20,BimmNaive,HLA-DQA2_BimmNaive,7.128877e-16
HLA-DQB2,6,HLA-DQB2,32723875,rs17612858,6,32620622,A,T,0.395551,⋯,0.05466370,8.199010e-50,0.1006890,0.01503450,2.124938e-11,3.754812e-14,20,BimmNaive,HLA-DQB2_BimmNaive,7.004266e-09
LAMB1,7,LAMB1,107564244,rs1131398,7,107569517,A,G,0.584623,⋯,0.07287180,4.439280e-32,-0.0799570,0.01437920,2.688348e-08,9.969587e-03,20,BimmNaive,LAMB1_BimmNaive,4.758895e-06
TUFM,16,TUFM,28853732,rs7187776,16,28857645,A,G,0.615571,⋯,0.01473370,3.420663e-67,-0.2497920,0.04560750,4.325910e-08,2.498464e-02,20,BimmNaive,TUFM_BimmNaive,7.008738e-06
ORMDL3,17,ORMDL3,38077294,rs8067378,17,38051348,A,G,0.491779,⋯,0.02442030,1.576034e-50,0.2259170,0.03306230,8.311311e-12,1.492448e-01,20,BimmNaive,ORMDL3_BimmNaive,3.055699e-09
